In [1]:
#-------------------------Imports---------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import random

from pycbc.waveform import get_td_waveform
from pycbc import types, fft, waveform

from scipy.stats import uniform
from scipy import interpolate
from scipy.integrate import quad
from scipy import constants

import os
#from astropy.cosmology import Planck15
from astropy.cosmology import Planck18
from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.table import Table

from multiprocessing import Pool
from multiprocessing import Process
import time
from tqdm import tqdm

###########################
# CONSTANTS
###########################
H0GLOB= 69.32#67.9 #69
Om0GLOB=0.298
Xi0Glob =1.
clight = 2.99792458* 10**5#km/s
cosmoglob = Planck18
cosmofast = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)
H0=cosmoglob.H(0).value
h=H0/100
#geometrization of masses
Msun=(1.98892)*(10**30)
solarmass_to_m=(constants.G*Msun)/((constants.c)**2)#G/c^2
Mpc_to_m=3.08567758128*(10**22) #this will be used later

#from lum distance to z
zz=np.linspace(0,20,10000)
distinterpol=cosmoglob.comoving_distance(zz).value*(1+zz)
z_of_dl=interpolate.interp1d(distinterpol,zz)

In [2]:
def phi_from_cart(x,y):
    if(x>0):
        phi=np.arctan(y/x)
    if ((x<0) and (y>=0)):
        phi=np.arctan(y/x)+np.pi
    if ((x<0) and (y<0)):
        phi=np.arctan(y/x)-np.pi
    return phi
def theta_from_cart(x,y,z):
    den=np.sqrt(x**2+y**2+z**2)
    theta=np.arccos(z/den)
    return theta

In [24]:
#-----------------Einstein Telescope Stuff------------------------------
#we will make two antenna functions, one is theresponse of a single interferometer
#the other will be the same as before, after a mean on psi

def ET_antenna_one_plus(cos_theta,phi,psi):
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_one_cross(cos_theta,phi,psi):
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
def ET_antenna_two_plus(cos_theta,phi,psi):
    phi=phi+(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_two_cross(cos_theta,phi,psi):
    phi=phi+(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
def ET_antenna_three_plus(cos_theta,phi,psi):
    phi=phi-(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_three_cross(cos_theta,phi,psi):
    phi=phi-(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
#constant is a prefactor for the signal, mass must be the redshifted mass, angles are vectors
def ET_SNR_quad(f,constant,mass,dist,cos_i,ds_cos_theta,ds_phi,ds_psi):
    dist=dist*Mpc_to_m
    cos_theta=np.random.choice(ds_cos_theta)
    phi=np.random.choice(ds_phi)
    psi=np.random.choice(ds_psi)
    hplus_squared=((constant/dist)*(mass**(5/6))*(((1+cos_i**2)/2)**2)*f**(-7/6))**2
    hcross_squared=((constant/dist)*(mass**(5/6))*(cos_i**2)*f**(-7/6))**2
    numerator01=hplus_squared*(ET_antenna_one_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_one_cross(cos_theta,phi,psi))**2
    denom=ET_sn(f)**2
    first=4*numerator01/denom
    numerator02=hplus_squared*(ET_antenna_two_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_two_cross(cos_theta,phi,psi))**2
    second=4*numerator02/denom
    numerator03=hplus_squared*(ET_antenna_three_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_three_cross(cos_theta,phi,psi))**2
    third=4*numerator03/denom
    ret=np.array([first,second,third])
    return ret
def signal_noise_toint(f,signal):
    #numerator=((constant/dist)*(mass**(5/6))*(((1+cos**2)/2)**2)*f**(-7/6))**2
    numerator=signal(f)**2
    denom=(ET_sn(f)**2)
    return (numerator/denom)


In [4]:
#----function that perform the SNR-----------------
#---use this function in a loop, you must provide the angles for each ds
#read it from the EVA (catalogues) series.
def ET_SNR_Evaluation(ds_theta,ds_phi,ds_psi,hp,hc):
    temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
    temp_cross=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hc))[0]
    first_prefactor=(ET_antenna_one_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_one_cross(ds_theta,ds_phi,ds_psi))**2
    firstSNR=(temp_plus+temp_cross)*first_prefactor*4
    second_prefactor=(ET_antenna_two_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_two_cross(ds_theta,ds_phi,ds_psi))**2
    secondSNR=(temp_plus+temp_cross)*second_prefactor*4
    third_prefactor=(ET_antenna_three_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_three_cross(ds_theta,ds_phi,ds_psi))**2
    thirdSNR=(temp_plus+temp_cross)*third_prefactor*4
    totSNR=np.sqrt(firstSNR+secondSNR+thirdSNR)
    return totSNR

In [5]:
#---------------Einstein Telescope Noise-----------------
ET_freq=np.loadtxt('ET-0000A-18_ETDSensitivityCurveTxtFile.txt',usecols=0)
ET_noise=np.loadtxt('ET-0000A-18_ETDSensitivityCurveTxtFile.txt',usecols=3)#This is the amplitude so must be squared
ET_sn=interpolate.interp1d(ET_freq,ET_noise)
ET_min_freq=ET_freq.min()
ET_max_freq=ET_freq.max()
print('Freq_min={}, Freq_max={}'.format(ET_min_freq,ET_max_freq))

Freq_min=1.0, Freq_max=10000.0


In [ ]:
#------------read the catalogue-------------------------------------
dat = Table.read('EVA01.fits', format='fits')
EVA = dat.to_pandas()#all good, is an only text fits
#print(EVA.columns)
ds_x=EVA['DS_x']
ds_y=EVA['DS_y']
ds_z=EVA['DS_z']
m1=EVA['M1']
m2=EVA['M2']
cos_inc=EVA['cos_orbital_incl']
inclin=np.arccos(cos_inc)
psi=EVA['psi']

In [ ]:
#----------Aproximant-------------------------------
#EVA['Aproximant']='IMRPhenomD'
#t = Table.from_pandas(EVA)
#t.write('EVA02.fits', overwrite=True)

In [ ]:
theta=np.zeros(EVA.shape[0])
phi=np.zeros(EVA.shape[0])
lum_distance=np.zeros(EVA.shape[0])
#SNR=np.zeros(EVA.shape[0])
for i in range(EVA.shape[0]):
    phi[i]=phi_from_cart(ds_x[i],ds_y[i])
    theta[i]=theta_from_cart(ds_x[i],ds_y[i],ds_z[i])
    lum_distance[i]=np.sqrt(ds_x[i]**2+ds_y[i]**2+ds_z[i]**2)

In [ ]:
start_time = time.time()
for i in tqdm(range(3)):
    FDhp,FDhc = waveform.get_fd_waveform(approximant="IMRPhenomD",
                             mass1=m1[i], mass2=m2[i],distance=lum_distance[i],inclination=inclin[i],
                                          delta_f=1.0/500,f_lower=ET_min_freq,f_final=ET_max_freq)
    hp_interpol=interpolate.interp1d(FDhp.sample_frequencies,abs(FDhp),fill_value="extrapolate")
    hc_interpol=interpolate.interp1d(FDhc.sample_frequencies,abs(FDhc),fill_value="extrapolate")
    SNR[i]=ET_SNR_Evaluation(theta[i],phi[i],psi[i],hp_interpol,hc_interpol)
end_time= time.time()
print("--- %s seconds ---" % ((time.time() - start_time)))

In [ ]:
print(SNR[0:3])

In [13]:
def Multi_SNR(i):
    FDhp,FDhc = waveform.get_fd_waveform(approximant="IMRPhenomD",
                             mass1=m1[i], mass2=m2[i],distance=lum_distance[i],inclination=inclin[i],
                                          delta_f=1.0/500,f_lower=ET_min_freq,f_final=ET_max_freq)
    hp_interpol=interpolate.interp1d(FDhp.sample_frequencies,abs(FDhp),fill_value="extrapolate")
    hc_interpol=interpolate.interp1d(FDhc.sample_frequencies,abs(FDhc),fill_value="extrapolate")
    temp=ET_SNR_Evaluation(theta[i],phi[i],psi[i],hp_interpol,hc_interpol)
    return temp
def merger_multi_SNR(i):
    if(tomerge[i]==1):
        FDhp,FDhc = waveform.get_fd_waveform(approximant="IMRPhenomD",
                             mass1=m1[i], mass2=m2[i],distance=lum_distance[i],inclination=inclin[i],
                                          delta_f=1.0/500,f_lower=ET_min_freq,f_final=ET_max_freq)
        hp_interpol=interpolate.interp1d(FDhp.sample_frequencies,abs(FDhp),fill_value="extrapolate")
        hc_interpol=interpolate.interp1d(FDhc.sample_frequencies,abs(FDhc),fill_value="extrapolate")
        temp=ET_SNR_Evaluation(theta[i],phi[i],psi[i],hp_interpol,hc_interpol)
    else:
        temp=mysnr[i]
    return temp

In [ ]:
print('I am doing the best I can, please wait...')
start_time = time.time()
with Pool(40) as p:
    mysnr=p.map(Multi_SNR, range(EVA.shape[0]))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#EVA['SNR']=mysnr
#t = Table.from_pandas(EVA)
#t.write('EVA02.fits', overwrite=True)

In [21]:
dat = Table.read('EVA02.fits', format='fits')
EVA= dat.to_pandas()#all good, is an only text fits

In [ ]:
EVA.head()

In [22]:
primarymass=np.array(EVA['M1'])
secondarymass=np.array(EVA['M2'])
snr=np.array(EVA['SNR'])
lines=np.zeros(len(primarymass))
totalmass=10
error=0.25
for i in range(len(primarymass)):
    m1=primarymass[i]
    m2=secondarymass[i]
    if ((m1+m2>(totalmass-error)) and (m1+m2)<(totalmass+error)):
        lines[i]=1
        #print(m1+m2)
lines=lines.astype(int)

In [32]:
#mydata=EVA
#mydata['count']=lines
#mydata=mydata[mydata['count']==1]
mydetected=mydata[mydata['SNR']>=8]
print(mydata.shape[0],mydetected.shape[0])

17674 17674


In [ ]:
detected=np.array(mydetected['count'])
total=np.array(mydata['count'])
redtotal=np.array(mydata['DS_redshift'])
reddetected=np.array(mydetected['DS_redshift'])

In [ ]:
steps=25
start=reddetected.min()
stop=reddetected.max()
percentage=[]
midpoint=[]
error=[]
binsize=(reddetected.max()-reddetected.min())/steps
for i in range(steps):
    start=reddetected.min()+i*binsize
    stop=reddetected.min()+(i+1)*binsize
    midpoint.append(start+(stop-start)/2)
    temp_detected=reddetected[reddetected<=stop]
    temp_detected=temp_detected[temp_detected>=start]
    temp_total=redtotal[redtotal<=stop]
    temp_total=temp_total[temp_total>=start]
    percentage.append(len(temp_detected)/len(temp_total))
    error.append(1/np.sqrt(len(temp_detected)))
percentage10=np.asarray(percentage)
midpoint10=np.asarray(midpoint)
error10=np.asarray(error)
#percentage20=np.asarray(percentage)
#midpoint20=np.asarray(midpoint)
#error20=np.asarray(error)
#percentage30=np.asarray(percentage)
#midpoint30=np.asarray(midpoint)
#error30=np.asarray(error)

In [ ]:
plt.rcParams.update({'font.size': 23})
fig,ax = plt.subplots()
fig.set_size_inches(18.5, 10.5)
m10, b10 = np.polyfit(midpoint10, percentage10, 1)
m20, b20 = np.polyfit(midpoint20, percentage20, 1)
m30, b30 = np.polyfit(midpoint30, percentage30, 1)


ax.errorbar(midpoint10, percentage10, yerr=0, fmt="o")
ax.plot(midpoint10[3:], percentage10[3:], 'yo', midpoint10[3:], m10*midpoint10[3:]+b10, '--b')
ax.scatter(midpoint10,percentage10,label='10M_sun',color='b')

#ax.errorbar(midpoint20, percentage20, yerr=0, fmt="o")
#ax.plot(midpoint20[3:], percentage20[3:], 'yo', midpoint20[3:], m20*midpoint20[3:]+b20, '--y')
#ax.scatter(midpoint20,percentage20,label='Efficency',color='r')

#ax.errorbar(midpoint30, percentage30, yerr=0, fmt="o")
#ax.plot(midpoint30[3:], percentage30[3:], 'yo', midpoint30[3:], m30*midpoint30[3:]+b10, '--r')
ax.scatter(midpoint30,percentage30,label='30M_sun',color='g')

plt.grid(axis='y', alpha=0.75)
plt.legend()
#plt.grid(axis='x', alpha=0.75)
plt.xlabel('Redshift')
plt.ylabel('Efficency')
plt.title('detection efficency')
plt.xscale('log')

In [9]:
mydata.head()

,Host_x,Host_y,Host_z,Host_redshift,DS_x,DS_y,DS_z,DS_redshift,cos_orbital_incl,M1,M2,q,psi,Aproximant,SNR,count
28,1473.370544,380.277651,2479.333496,0.808604,1473.370544,380.277651,2479.333496,0.808604,0.081533,6.641128,3.223255,0.485348,5.727144,b'IMRPhenomD',20.975862,1
72,592.358854,1725.526178,2265.015559,0.808363,592.358854,1725.526178,2265.015559,0.808363,-0.659711,6.682928,3.125461,0.467678,0.854460,b'IMRPhenomD',46.315407,1
77,867.047301,2537.836331,1096.379431,0.804409,867.047301,2537.836331,1096.379431,0.804409,-0.993184,7.105248,2.903938,0.408703,4.278400,b'IMRPhenomD',111.768054,1
115,539.991464,1683.082330,2322.028954,0.811904,539.991464,1683.082330,2322.028954,0.811904,0.660126,6.980936,2.778118,0.397958,1.134512,b'IMRPhenomD',43.047301,1
166,463.535184,341.658635,2868.854033,0.814716,464.456642,342.540150,2866.198364,0.813869,0.534297,8.621414,1.359953,0.157741,0.017572,b'IMRPhenomD',16.274057,1


In [26]:
#------------read the catalogue-------------------------------------
ds_x=np.asarray(mydata['DS_x'])
ds_y=np.asarray(mydata['DS_y'])
ds_z=np.asarray(mydata['DS_z'])
m1=np.asarray(mydata['M1'])
m2=np.asarray(mydata['M2'])
cos_inc=np.asarray(mydata['cos_orbital_incl'])
inclin=np.zeros(len(cos_inc))#np.arccos(cos_inc)
psi=np.asarray(mydata['psi'])

In [27]:
theta=np.zeros(mydata.shape[0])
phi=np.zeros(mydata.shape[0])
lum_distance=np.zeros(mydata.shape[0])
#SNR=np.zeros(EVA.shape[0])
for i in range(mydata.shape[0]):
    phi[i]=phi_from_cart(ds_x[i],ds_y[i])
    theta[i]=theta_from_cart(ds_x[i],ds_y[i],ds_z[i])
    lum_distance[i]=np.sqrt(ds_x[i]**2+ds_y[i]**2+ds_z[i]**2)

In [29]:
print('I am doing the best I can, please wait...')
start_time = time.time()
with Pool(40) as p:
    mynewsnr=p.map(Multi_SNR, range(mydata.shape[0]))
print("--- %s seconds ---" % (time.time() - start_time))

I am doing the best I can, please wait...


<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges

<ipython-input-4-14f25aa611d6>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_cross=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hc))[0]
<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subrange

<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges

<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
<ipython-input-4-14f25aa611d6>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges

<ipython-input-4-14f25aa611d6>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_cross=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hc))[0]
<ipython-input-4-14f25aa611d6>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subrange

<ipython-input-4-14f25aa611d6>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_cross=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hc))[0]
<ipython-input-4-14f25aa611d6>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subrange

--- 2209.3961791992188 seconds ---


In [30]:
mydata.drop('SNR', inplace=True, axis=1)
mydata.insert(loc=14, column='SNR', value=mynewsnr)

In [31]:
mydata.head()

,Host_x,Host_y,Host_z,Host_redshift,DS_x,DS_y,DS_z,DS_redshift,cos_orbital_incl,M1,M2,q,psi,Aproximant,SNR,count
28,1473.370544,380.277651,2479.333496,0.808604,1473.370544,380.277651,2479.333496,0.808604,0.081533,6.641128,3.223255,0.485348,5.727144,b'IMRPhenomD',58.178546,1
72,592.358854,1725.526178,2265.015559,0.808363,592.358854,1725.526178,2265.015559,0.808363,-0.659711,6.682928,3.125461,0.467678,0.854460,b'IMRPhenomD',67.195008,1
77,867.047301,2537.836331,1096.379431,0.804409,867.047301,2537.836331,1096.379431,0.804409,-0.993184,7.105248,2.903938,0.408703,4.278400,b'IMRPhenomD',112.533819,1
115,539.991464,1683.082330,2322.028954,0.811904,539.991464,1683.082330,2322.028954,0.811904,0.660126,6.980936,2.778118,0.397958,1.134512,b'IMRPhenomD',62.422742,1
166,463.535184,341.658635,2868.854033,0.814716,464.456642,342.540150,2866.198364,0.813869,0.534297,8.621414,1.359953,0.157741,0.017572,b'IMRPhenomD',27.536031,1
